# Generate Initial Dataset
While it isn't necessary to start with a learned potential when doing on-the-fly learning, it will accelerate the process.

In [ ]:
from cascade.calculator import make_calculator
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.nptberendsen import NPTBerendsen
from ase.io import read, Trajectory
from ase import units
from pathlib import Path

Configuration

In [ ]:
starting_strc = 'structures/si-perfect-2x2x2.vasp'
timesteps = 512

In [ ]:
starting_strc = Path(starting_strc)

## Load the Structure and make a Calculator
The two ingredients we need to run the dynamics

In [ ]:
atoms = read(starting_strc)

In [ ]:
calc = make_calculator('blyp')

## Get the starting point
Either a thermalized structure or the last checkpoint

In [ ]:
run_dir = Path('init-run') / starting_strc.name
traj_path = run_dir / 'md.traj'

In [ ]:
if traj_path.exists():
    atoms = read(traj_path, -1)
else:
    run_dir.mkdir(exist_ok=True, parents=True)
    MaxwellBoltzmannDistribution(atoms, temperature_K=300)

Run dynamics

In [ ]:
with Trajectory(traj_path, mode='a', atoms=atoms) as traj:
    atoms.calc = calc
    npt = NPTBerendsen(
        atoms,
        logfile=run_dir / 'md.log',
        timestep=0.3 * units.fs,
        taut=10. * units.fs,  # Small because the structure starts out very unrelaxed
        temperature_K=300,
        pressure_au=0,
        compressibility_au=4.57e-5 / units.bar
    )
    npt.attach(traj.write, 1)
    npt.run(timesteps)